In [ ]:
# Configure AWS profile for local development
%env AWS_PROFILE=platform-developer

In [ ]:
import sys
import os
import logging

# Add src to path if not present
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oai_pmh_client").setLevel(logging.INFO)

from datetime import datetime, timezone

# Select which adapter to use - change this to "folio" or "axiell"
ADAPTER = "folio"  # Options: "axiell", "folio"

if ADAPTER == "axiell":
    from adapters.axiell.runtime import AXIELL_CONFIG as ADAPTER_CONFIG
    from adapters.axiell import config
elif ADAPTER == "folio":
    from adapters.folio.runtime import FOLIO_CONFIG as ADAPTER_CONFIG
    from adapters.folio import config
else:
    raise ValueError(f"Unknown adapter: {ADAPTER}")

# Construct the OAI-PMH client
client = ADAPTER_CONFIG.build_oai_client()

# Configure start and end date
start_date = datetime(2025, 11, 28, 23, 46, tzinfo=timezone.utc)
end_date = datetime(2025, 11, 29, 0, 0, tzinfo=timezone.utc)

print(f"Using {ADAPTER.upper()} adapter")
print(f"Querying from {start_date} to {end_date}...")

# Make queries to the OAI-PMH service
records = client.list_records(
    metadata_prefix=config.OAI_METADATA_PREFIX,
    set_spec=config.OAI_SET_SPEC,
    from_date=start_date,
    until_date=end_date
)

# Iterate and print the first few records
for i, record in enumerate(records):
    print(f"Record {i}: {record.header.identifier}")
    if i >= 4:
        break